In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
dbutils.fs.ls("abfss://landing@salesmarketingstorage1.dfs.core.windows.net")

In [0]:
df = spark.read \
.options(header=True) \
.csv("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/order_payments")

df.show()

In [0]:
df_order_payments = (
    spark.readStream 
    .format("cloudFiles").option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("mode", "PERMISSIVE")
    .option("cloudFiles.schemaLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/bronze/order_payments")
    .load("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/order_payments/")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/order_payments") 
    .table("bronze.order_payments")
)

In [0]:
%sql 
describe sales_marketing_ws.bronze.order_payments;

In [0]:
%sql
SELECT * FROM cloud_files_state('abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/order_payments');

In [0]:
%sql
select * from sales_marketing_ws.bronze.order_payments limit 10;